# 使用普通神经网络包含节点特征

In [2]:
from torch_geometric.datasets import Planetoid

In [7]:
dataset = Planetoid(root=".", name="Cora")

FSTimeoutError: 

In [6]:
data = dataset[0]
print(dataset)
print(len(dataset))
print(data.x.shape[0])
print(dataset.num_features)
print(dataset.num_classes)


NameError: name 'dataset' is not defined